## Assignment 3

### Part A3.1: Access PMU from python

* http://neocontra.blogspot.com/2013/05/user-mode-performance-counters-for.html
* https://www.cprogramming.com/tutorial/shared-libraries-linux-gcc.html
*https://stackoverflow.com/questions/3247373/how-to-measure-program-execution-time-in-arm-cortex-a8-processor/3250835#3250835

In [63]:
import ctypes

In [65]:
SUCCESS = 0
FAILURE = -1
CPU_FHz = 650e6

Now we can import our shared library

In [66]:
_libInC = ctypes.CDLL('./libPMULib.so')

In [46]:
counter = _libInC.my_init_counters(1,0)
print(counter)

1


In [47]:
a = _libInC.my_get_cyclecount()
print(a)

769503020


### Part A3.2: Comparing and Gathering Data

In [87]:
import multiprocessing
import os
import time

In [88]:
# Program to calculate the Fibonacci sequence up to n-th term
def recur_fibo(n):
   if n <= 1:
       return n
   else:
       return(recur_fibo(n-1) + recur_fibo(n-2))

In [131]:
def performance_metrics_calculator(_id, num_fib):
    #Assign the process to specific CPU
    os.system("taskset -pc {} {}".format(_id,os.getpid()))
    
    #take python module time
    tic = time.time()
    
    #get cycle count
    cpu_time1 = 64* ctypes.c_uint32(_libInC.my_get_cyclecount()).value
    print("Cycle count 1 = {}".format(cpu_time1))

    if cpu_time1 == 0:
        print("1stcycle count is 0 invalid read");
    
    # Run recur fib program
    recur_fibo(num_fib)
    
    #get cycle count 
    cpu_time2 = 64* ctypes.c_uint32(_libInC.my_get_cyclecount()).value
    print("Cycle count 2 = {}".format(cpu_time2))
    
    if cpu_time2 == 0:
        print("2nd cycle count is 0 invalid read");

    if (cpu_time2 - cpu_time1) < 0:
        print("2nd cycle count is less than cpu_time1");
        print("Overflown counter value = {}".format(cpu_time2))
        cpu_time2 = 64*(2**32+cpu_time2)
        print("New Cycle count 2 = {}".format(cpu_time2))
    
    tac = time.time()

    time_cycleCount = (abs(cpu_time2-cpu_time1) / CPU_FHz)

    print("Timer module value ={} ".format(tac-tic))
    print("Cycle count value = {}".format(time_cycleCount))
    
    
    

In [132]:
def main():
    nterms = int(input("Enter How many terms of Fibonacci? "))
    # check if the number of terms is valid
    if nterms <= 0:
        print("Please enter a positive integer")
        return FAILURE
    
    cpu_id = int(input("Enter CPU ID(0/1) for CPU selection"))
    print("CPU ID Entered = ", cpu_id)
    # check if cpu id is valid
    if cpu_id < 0 or cpu_id > 1:
        print("unsupported CPU id Entered")
        return FAILURE
    
    # create a process
    p1 = multiprocessing.Process(target=performance_metrics_calculator, args=(cpu_id,nterms)) # the first arg defines which CPU to run the 'target' on
    # start the process
    p1.start() 
    p1Name = p1.name # get process1 name
    p1.join() # wait for process1 to finish
    print('Process with name, {}, is finished'.format(p1Name))

### Main function to handle input and spawn separate process

In [134]:
if __name__ == '__main__':
    _libInC.my_init_counters(1, 1);
    result = main()
    
    if result == FAILURE:
        print("Program exited, exit code = FAILURE")
    else:
        print("Program exited successfully")

Enter How many terms of Fibonacci? 35
Enter CPU ID(0/1) for CPU selection0
CPU ID Entered =  0
Cycle count 1 = 79214848
Cycle count 2 = 26850457088
Timer module value =41.213409662246704 
Cycle count value = 41.18652652307692
Process with name, Process-50, is finished
Program exited successfully
